This group of exercises is about linked lists, so I'll copy over an earlier implementation of a linked list (from [here](https://github.com/ResidentMario/data-structures-js/blob/master/linked_list.js)) to test with.

In [1]:
class _Node {
    constructor(value, next=null) {
        this.value = value;
        this.next = next;
    }
}

class LinkedListADT {

    /**
     * Singely linked list ADT constructor.
     * @constructor
     */
    constructor() {
        this.head = null;
        this.tail = null;
        this.length = 0;
    }

    append(v) {
        let n = new _Node(v);
        if (!this.head) {
            this.head = n;
            this.tail = n;
        } else {
            this.tail.next = n;
            this.tail = n;
        }
        this.length += 1;
    }

    get_node(idx) {
        if (idx >= this.length || idx < 0) { throw SyntaxError("Attempting a get that is out of range."); }
        else if (idx === 0) { return this.head; }
        else if (idx === this.length - 1 ) { return this.tail; }
        else {
            // Iterate through, skipping the first node because we know we haven't hit that case.
            let c = 1;
            let n_p = this.head.next;
            while (c  < idx) {
                n_p = n_p.next;
            }
            return n_p;
        }
    }

    get(idx) {
        return this.get_node(idx).value;
    }

    /**
     * Finds and returns the node with the given value.
     * @param v - The value to be sought out.
     * @returns {*}
     */
    find(v) {
        let idx = 0;
        let n = this.head;
        while (idx <= this.length) {
            if (n.value === v) {
                return n;
            }
                n = n.next;
                idx += 1;
        }
        throw Error(`The value ${v} is not contained in the list.`)
    }

    /**
     * Linked list ADT insert operation.
     * @param v - The value to store.
     * @param idx - The index of the linked node to store the value at.
     */
    insert(v, idx) {
        if (idx > this.length || idx < 0) { throw SyntaxError("Attempting an insert that is out of range."); }

        if (idx === 0) {
            this.head = new _Node(v, this.head);
        } else if (idx === this.length) {
            let n_p = this.tail;
            let new_tail = new _Node(v);
            n_p.next = new_tail;
            this.tail = new_tail;
        } else {
            let n_p = this.get_node(idx - 1);
            let n_n = n_p.next;
            n_p.next = new _Node(v, n_n);
        }
        this.length += 1;

    }

    remove(idx) {
        if (idx > this.length || idx < 0) { throw SyntaxError("Attempting a remove that is out of range."); }

        if (idx === 0) {
            let n = this.head;
            this.head = n.next;
            return n;
        } else if (idx === this.length - 1) {
            let n = this.get_node(this.length - 2);
            this.tail = n;
            return n;
        }
    }
    
    value_list() {
        if (this.length === 0)  return [];
        
        let out = [];
        let pointer = this.get_node(0);
        
        while (true) {
            out.push(pointer.value);
            
            if (pointer.next === null)  break;
            else  pointer = pointer.next;
        }
        
        return out;
    }
}

## 2.1

Q: Remove duplicates from an unsorted linked list.

A simple solution.

In [15]:
function dedupe(ll) {
    if (!ll.length || ll.length === 1)  return ll;
    
    let pointer = ll.get_node(0);
    let out = new LinkedListADT();
    let out_vals = new Set();
    
    while (true) {
        if (!out_vals.has(pointer.value)) {
            out.append(pointer.value);
            out_vals.add(pointer.value);
        }
        
        if (pointer.next === null) {
            break;
        } else {
            pointer = pointer.next;
        }
    }
    
    return out;
}

In [16]:
let ll = new LinkedListADT();
ll.append(1); ll.append(2); ll.append(3);
assert.deepEqual(dedupe(ll).value_list(), [1, 2, 3])

In [20]:
let ll2 = new LinkedListADT();
ll2.append(1); ll2.append(1); ll2.append(1);
assert.deepEqual(dedupe(ll2).value_list(), [1]);

In [21]:
let ll3 = new LinkedListADT();
assert.deepEqual(dedupe(ll3).value_list(), []);

This solution is $O(n)$ time and $O(m)$ space, where $m$ is the number of unique nodes in the linked list. We can get reduce the space requirement by a constant fractional factor by optimizing this algorithm so that it performs the deduplication in-place, without requiring the construction of a new object:

In [1]:
function dedupe(ll) {
    if (!ll.length || ll.length === 1)  return ll;
    
    let pointer_0 = ll.get_node(0);
    let pointer_1 = ll.get_node(1);    
    let out_vals = new Set();
    out_vals.add(pointer_0.value);
    
    while (true) {
        if (out_vals.has(pointer_1.value)) {
            if (pointer_1.next === null) {
                pointer_0.next = null;
                ll.tail = pointer_0;
                break;
            } else {
                pointer_0.next = pointer_1.next;
                
                pointer_1 = pointer_1.next;
            }
        } else {
            out_vals.add(pointer_1.next);
            
            if (pointer_1.next === null) {
                break;
            }
            
            pointer_0 = pointer_1;       
            pointer_1 = pointer_1.next;
        }
    }
    
    return ll;
}

In [3]:
let ll4 = new LinkedListADT();
ll4.append(1); ll4.append(2); ll4.append(3);
assert.deepEqual(dedupe(ll4).value_list(), [1, 2, 3]);

In [12]:
let ll5 = new LinkedListADT();
ll5.append(1); ll4.append(1); ll4.append(1);
assert.deepEqual(dedupe(ll5).value_list(), [1]);

## 2.4

Q: Partition a linked list by comparing against a value $X$ which may or may not be in the list.

In [42]:
function partition(ll, v) {
    if (!ll.length || ll.length === 1)  return ll;
    
    let pointer = ll.head;
    let out = new LinkedListADT();

    out.append(-1);
    out.append(v);
    let prior_dummy = out.head;
    let v_dummy = out.head.next;
    
    while (true) {
        if (pointer.value > v) {
            out.append(pointer.value);
        } else if (pointer.value < v) {
            out.head = new _Node(pointer.value, next=out.head);
            prior_dummy = prior_dummy.next;
        } else {  // pointer.value === v
            v_dummy.next = new _Node(v, next=v_dummy.next);
        }
        
        if (pointer.next === null) {
            break;
        } else {
            pointer = pointer.next;
        }
    }
    
    // Wrong, but whatever.
    return out;
}

In [43]:
partition(ll4, 2).value_list();

[ 1, -1, 2, 3 ]

Lesson learned here: I should have done a proper analysis on this one first.

## 2.5

Q: Sum together two numbers stored as per-digit, ascending digit order linked lists. For example, sum `LL([1,2,3]) + LL([3,4,5]) = LL([4,6,7])`

```python
function sumLL(l1, l2):
    if l1.length == 0 return l2
    else if l2.length == 0 return l1

    l1_digit_node, l2_digit_node = l1.head, l2.head
    carry = False
    
    while True:
        if l1_digit_node and not l2_digit_node:
            if carry:
                l1_digit_node.value += 1
                return l1
            else:
                return l1
        else if not l1_digit_node and l2_digit_node:
            if carry:
                l1.tail.next = l2_digit_node.value + 1
            else:
                l1.tail.next = l2_digit_node.value
        else:
            if carry:
                l1_digit_node.value = ((l1_digit_node.value + l2_digit_node.value + 1) % 10)
            else:
                l1_digit_node.value = (l1_digit_node.value + l2_digit_node.value) % 10

        carry = l1_digit_node.value + l2_digit_node.value > 9
                
        l1_digit_node = l1_digit_node.next if l1_digit_node else l1_digit_node
        l2_digit_node = l2_digit_node.next if l2_digit_node else l2_digit_node
```

In [59]:
function sumLL(l1, l2) {
    if (l1.length === 0)  return l2;
    if (l2.length === 0)  return l1;
    
    let [l1_digit_node, l2_digit_node] = [l1.head, l2.head];
    let carry = false;
    
    while (true) {
        if (l1_digit_node === null & l2_digit_node === null) {
            if (carry)  l1.append(1);
            break
        } else if (l1_digit_node !== null & l2_digit_node === null) {
            if (carry) { 
                if (l1.tail.value == 9) { l1.tail.value = 0; l1.append(1) }
                else l1.tail.value += 1; 
            }
            break
        } else if (l1_digit_node === null & l2_digit_node !== null) {
            l1.append(l2_digit_node.value);
            if (carry)  l1.tail.value += 1;
            
            if (l1_digit_node.value > 9) { l1_digit_node.value = l1_digit_node.value % 10; carry = true; }
            else  carry = false;
        } else {
            l1_digit_node.value = l1_digit_node.value + l2_digit_node.value;
            if (carry)  l1_digit_node.value += 1;

            if (l1_digit_node.value > 9) { l1_digit_node.value = l1_digit_node.value % 10; carry = true; }
            else  carry = false;
        }
        
        l1_digit_node = l1_digit_node.next ? l1_digit_node.next : null; 
        l2_digit_node = l2_digit_node.next ? l2_digit_node.next : null;        
    }
    
    return l1;
}

In [61]:
let l1 = new LinkedListADT();
let l2 = new LinkedListADT();

In [37]:
l1.head = null;
l1.append(1); l1.append(2); l1.append(3);
l2.head = null;
l2.append(4); l2.append(5); l2.append(6);
assert.deepEqual(sumLL(l1, l2).value_list(), [5, 7, 9]);

In [38]:
l1.head = null;
l1.append(0); l1.append(0); l1.append(5);
l2.head = null;
l2.append(0); l2.append(0); l2.append(5);
assert.deepEqual(sumLL(l1, l2).value_list(), [0, 0, 0, 1]);

In [39]:
l1.head = null;
l1.append(5); l1.append(5); l1.append(5);
l2.head = null;
l2.append(5); l2.append(5); l2.append(5);
assert.deepEqual(sumLL(l1, l2).value_list(), [0, 1, 1, 1]);

In [60]:
l1.head = null;
l1.append(0); l1.append(9); l1.append(9);
l2.head = null;
l2.append(0); l2.append(9);
assert.deepEqual(sumLL(l1, l2).value_list(), [0, 8, 0, 1]);

Comments:
* More logic errors than I care to remember.
* I could have simplified the code with no loss of generality by swapping `l1` and `l2` when `l1` is shorter than `l2`. Only realized this in retrospect.

## 2.6

Q: Check if a linked list is a palindrome.

This is trivial if the linked list is doubly linked. It's an annoying exercise if it's singly linked.

In [ ]:
function palindrome(ll) {
    if (ll.length <= 1)  return true;

    let curr_node = ll.head;
    let curr_node_idx = 0;
    let is_even = ((ll.length % 2) === 0);
    let up_to = Math.floor(ll.length / 2);
    let up_to_arr = Array(up_to);
    
    while (true) {
        if (curr_node_idx < up_to) {
            up_to_arr[curr_node_idx] = curr_node;
            curr_node = curr_node.next;
            curr_node_idx += 1;
        } else if (curr_node_idx === up_to) {
            // Skip if odd.
            if (!is_even) {
                curr_node = curr_node.next;
                curr_node_idx += 1;
            } else {
                up_to_arr[curr_node_idx] = curr_node;
                curr_node = curr_node.next;
                curr_node_idx += 1;
            }
        } else {  // curr_node_idx > up_to
            let expected = up_to_arr.pop()
        }
    }
}